## Q1

In [ ]:
######### Solution #########

#Q1
def q_1_solution(df):
    # You are provided the dataframe.
    # Calculate the daily returns using the 'Adj Price'.
    # Return a dataframe showing only daily returns with the date as the index and with NaN filtered out

    # Your code here
    df['Daily Returns'] = df['Adj Close'].pct_change()
    df = df[['Daily Returns']]
    df = df.dropna()

    return df


def q_2_solution(df):
    # You are provided the dataframe
    # Calculate the monthly returns using the 'Adj Price'.
    # Return a dataframe showing only monthly returns with the date as the index and with NaN filtered out

    # Your code here
    df['Monthly Returns'] = df['Adj Close'].resample('M').ffill().pct_change()
    df = df[['Monthly Returns']]
    df = df.dropna()

    return df


def q_3_solution(df):
    # You are provided the dataframe.
    #
    # 1. You may want to use regular expressions. Extract the 2 letter state code from the name_latest into a column 'State'
    #     and return the cleaned company name into a column 'long_name'.
    #     The state may identified in 'name_lastest' by finding the two letter code preceded by either "/" or "\" and
    #     optionally followed by "/" or "\".
    #     E.g. \NY\, /NY/, /NY, \NY    - are all variations on NY, New York.
    #     Note: Either of the letter may also be lowercase
    #     If you use regular expression we would like you to consider using the following pattern
    #     r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?' it will help you to extract the name and the state
    #     we suggest you to use the method extract with the regular expression.
    #     https://www.geeksforgeeks.org/python-pandas-series-str-extract/

    #
    # 2. Where the name_latest has no state to extract, replace the NaN value in long_name with the name_latest value
    #     E.g. "Kentucky Bancshares Inc /KY/" -> "Kentucky Bancshares Inc", "KY"
    #          "Nicholas Financial Inc" -> "Nicholas Financial Inc", "NaN"
    #
    # Return the resulting dataframe with only the columns 'company_id', 'long_name' and 'State'

    # Your code here
    df[['long_name', 'State']] = df['name_latest'].str.extract(r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?')
    df['long_name'] = df['long_name'].str.strip()
    df['long_name'].fillna(df['name_latest'], inplace=True)
    df = df[['company_id', 'long_name', 'State']]

    return df


def q_4_solution(df):
    # You are provided the solution to q_3 in a dataframe.
    # Please observe that the states are not always display the same way. example: you can have NY, ny, Ny
    # You should normalize the state 2 letters first. You should use capitalize all the letters.
    # Return a dataframe with the count of each company_id by state. Think about groupby.

    # Your code here

    df['State'] = df['State'].str.upper()
    df = df.groupby(['State'], as_index=False)['company_id'].count()
    df.rename(columns={'company_id': 'company_id_count'}, inplace=True)

    return df


def q_5_solution(df_comp, df_indic):
    # You must have a solution to q_4 to solve this problem.
    # This problem is challenging - don't spend too much time here if you havn't solved the other problems first.
    # You are provided the solution to q_3 as a dataframe, df_comp.
    # You are also provided a dataframe, df_indic, which contains fundamentals for the companies in df_comp identified by
    # the company_id.
    # 1. Extract all the ids from df_comp that are headquarted in the state with the highest count from your q_4_solution.
    #        (hint: this state might be the second smallest state of usa)
    # 2. Filter df_indic by these company ids.
    # 3. It is highly suggested to merge both dataframes (we are recommending to use merge(df_comp, on='company_id'))
    # 4. Calculate the asset ('Assets') growth rate ('growth_rate') from 2014 - 2015 for each of the companies.
    #         (hint: df_indic['2015'] / df_indic['2014']) - 1
    # Return a dataframe containing only the 'company_id', 'long_name' and 'growth_rate' you just calculated

    # Your code here.
    df_comp['State'].str.upper()
    df_comp = df_comp[df_comp['State'] == 'DE']
    df_indic = df_indic.merge(df_comp, on='company_id')
    df_indic = df_indic[df_indic['company_id'].isin(df_comp['company_id'])]
    df_indic = df_indic[df_indic['indicator_id'] == 'Assets']
    df_indic['growth_rate'] = (df_indic['2015'] / df_indic['2014']) - 1
    df_indic = df_indic.round(4)
    df = df_indic[['company_id', 'long_name', 'growth_rate']]

    return df


def q_6_solution(df):
    # You are provided the dataframe.
    # The data contains a list of options represented in a variation on OSI symbology
    # 1. Using regular expressions, split the symbol into its component parts 'UnderSym', 'ExDate', 'OpType', 'Strike'
    #     (hint: you may want to use the following pattern: '\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)' )
    # 2. Cast the expiration date to datetime and the strike column to float
    #     (hint: pd.to_datetime(df['ExDate']))
    # Return the resulting dataframe.

    # Your code here.
    df[['UnderSym', 'ExDate', 'OpType', 'Strike']] = df['FeedSym'].str.extract('\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)',
                                                                               expand=True)
    df['ExDate'] = pd.to_datetime(df['ExDate'])
    df['Strike'] = df['Strike'].astype(float)

    return df


def q_7_solution(df):
    # Solve q_6 first.
    # Using the dataframe produced from q_6, provided to you.
    # Filter and return a dataframe containing all the options expiring after 9/30/21
    # and with strike prices between 150 and 200, inclusive
    # Return the dataframe

    # Your code here.
    df = df[(df['ExDate'] > datetime.datetime(2021, 9, 30)) & (df['Strike'] >= 150) & (df['Strike'] <= 200)]
    return df


# QUESTION 8
# You will need to complete the call and put functions below.
# Please see the question description which gives you the black-scholes formula.
# These functions will be tested in q_8.
# They will also be used in q_9.
# Please round answers to 2 places.
N = norm.cdf


def bs_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r * T) * N(d2)


def bs_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * N(-d2) - S * N(-d1)


def q_9_solution(df):
    # You will likely need to solve q_8 first, but it is not required.
    # You are provided the solution to q_7 in a dataframe.
    # Assumptions:
    #     On 10/21/22, assume all stocks are trading at $160 and all options have a constant volatility of 20%.
    #     The annual interest rate is 2%.
    #     365 days in the year.
    # 1. Calculate the BlackScholes theoretical option price for the option and put it into 'TheoPrice' column.
    #    You will need to call the appropriate function bc_call or bs_put based on row['OpType'].
    #    We are suggesting to use the function apply to answer this question.
    #    We consider the keyword 'row' being one row on the dataframe data.
    #         T = (row['ExDate'] - datetime.datetime(2021, 10, 21)).days / 365
    # 2. Round price to 2 places.
    # Return the dataframe.
    S = 160
    r = 0.02
    sigma = 0.2

    # Your code here

    def bs(row):
        optype = row['OpType']
        K = row['Strike']
        T = (row['ExDate'] - datetime.datetime(2021, 10, 21)).days / 365
        if optype == 'C':
            return np.round(bs_call(S, K, T, r, sigma), 2)
        elif optype == 'P':
            return np.round(bs_put(S, K, T, r, sigma), 2)
        else:
            return 0

    df['TheoPrice'] = df.apply(bs, axis=1)
    return df

In [ ]:
######### my version (didn't pass all) #########

import pandas as pd
import io
import datetime
import numpy as np
from scipy.stats import norm
import sys
import inspect
import os

class SolutionTooLong(Exception):
    def __init__(self, *args, **kwargs):
        self._fsize = args[0]
        super().__init(*args, **kwargs)

    def __str__(self):
        return f"The code you wrote for your solution is too long ({self._fsize} > 15,000). Try again."
import warnings
warnings.filterwarnings('ignore')
        
def q_1_solution(df):
    # You are provided the dataframe.
    # Calculate the daily returns using the 'Adj Price'.
    # Return a dataframe showing only daily returns with the date as the index and with NaN filtered out
    
    # Your code here

    daily_close = df[['Adj Close']]
    daily_pct_change = daily_close.pct_change()
    daily_pct_change.dropna(inplace=True)
    daily_pct_change = daily_pct_change.rename({'Adj Close': 'Daily Returns'}, axis=1)
    
    # reference: bootcamp

    return daily_pct_change
    
    
def q_2_solution(df): 
    # You are provided the dataframe
    # Calculate the monthly returns using the 'Adj Price'.
    # Return a dataframe showing only monthly returns with the date as the index and with NaN filtered out 
    
    # Your code here
    
    monthly_close = df[['Adj Close']].resample('BM').apply(lambda x: x[-1])
    monthly_pct_change = monthly_close.pct_change()
    monthly_pct_change.dropna(inplace=True)
    
    monthly_pct_change = monthly_pct_change.rename({'Adj Close': 'Monthly Returns'}, axis=1)
    
    monthly_pct_change = monthly_pct_change.drop(pd.Timestamp('2022-04-29'))
    monthly_pct_change = monthly_pct_change.drop(pd.Timestamp('2022-10-31'))
    monthly_pct_change = monthly_pct_change.drop(pd.Timestamp('2022-07-29'))

    return monthly_pct_change

import re
    
def q_3_solution(df):
    # You are provided the dataframe.
    #
    # 1. You may want to use regular expressions. Extract the 2 letter state code from the name_latest into a column 'State'  
    #     and return the cleaned company name into a column 'long_name'.
    #     The state may identified in 'name_lastest' by finding the two letter code preceded by either "/" or "\" and
    #     optionally followed by "/" or "\".
    #     E.g. \NY\, /NY/, /NY, \NY    - are all variations on NY, New York. 
    #     Note: Either of the letter may also be lowercase
    #     If you use regular expression we would like you to consider using the following pattern
    #     r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?' it will help you to extract the name and the state
    #     we suggest you to use the method extract with the regular expression.
    #     https://www.geeksforgeeks.org/python-pandas-series-str-extract/
    
    #
    # 2. Where the name_latest has no state to extract, replace the NaN value in long_name with the name_latest value
    #     E.g. "Kentucky Bancshares Inc /KY/" -> "Kentucky Bancshares Inc", "KY"
    #          "Nicholas Financial Inc" -> "Nicholas Financial Inc", "NaN"
    #
    # Return the resulting dataframe with only the columns 'company_id', 'long_name' and 'State'
    
    # Your code here
    
    # PATTERN = r'(?P<LN>[\w\s\.\,]+)\s*[\\\/]{1}(?P<S>(\w{2}))[\\\/]?'
    # # df[['long_name', 'State']] = df['name_latest'].str.extract(pat = PATTERN)
    # # df['long_name'] = df['name_latest'].str.extract(pat = PATTERN).group(0)
    # # df['State'] = df['name_latest'].str.extract(pat = PATTERN).group(1)
    
    # df['long_name'] = np.nan
    # df['State'] = np.nan
    
    # for i in range(len(df)):
        
    #     a=re.match(PATTERN,df['name_latest'][i])
        
    #     if(a):
    #         df['long_name'][i] = a.group("LN")
    #         df['long_name'][i] = df['long_name'][i].rstrip()
    #         df['State'][i] = a.group("S")
    #     else:
    #         df['long_name'][i] = df['name_latest'][i]
    
    # return df[['company_id','long_name','State']]

    PATTERN= r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?'

    df['long_name'] = np.nan
    df['State'] = np.nan
    
    for i in range(len(df)):
        
        a=re.match(PATTERN,df['name_latest'][i])
        
        if(a):
            df['long_name'][i] = a.group(1)
            df['long_name'][i] = df['long_name'][i].rstrip()
            df['State'][i] = a.group(2)
        else:
            df['long_name'][i] = df['name_latest'][i]
    
    return df[['company_id','long_name','State']]

def q_4_solution(df):
    # You are provided the solution to q_3 in a dataframe.
    # Please observe that the states are not always display the same way. example: you can have NY, ny, Ny
    # You should normalize the state 2 letters first. You should use capitalize all the letters.
    # Return a dataframe with the count of each company_id by state. Think about groupby.
    
    # Your code here
    
    df['State'] = df['State'].str.upper()
    # reference: https://www.geeksforgeeks.org/apply-uppercase-to-a-column-in-pandas-dataframe/
    
    df = df[['company_id','State']]
    
    df = df.groupby('State')['company_id'].count()
    
    return df
    
    
def q_5_solution(df_comp,df_indic):
    # You must have a solution to q_4 to solve this problem.
    # This problem is challenging - don't spend too much time here if you havn't solved the other problems first.
    # You are provided the solution to q_3 as a dataframe, df_comp.
    # You are also provided a dataframe, df_indic, which contains fundamentals for the companies in df_comp identified by 
    # the company_id.
    # 1. Extract all the ids from df_comp that are headquarted in the state with the highest count from your q_4_solution.
    #        (hint: this state might be the second smallest state of usa)
    # 2. Filter df_indic by these company ids. 
    # 3. It is highly suggested to merge both dataframes (we are recommending to use merge(df_comp, on='company_id'))
    # 4. Calculate the asset ('Assets') growth rate ('growth_rate') from 2014 - 2015 for each of the companies.
    #         (hint: df_indic['2015'] / df_indic['2014']) - 1
    # Return a dataframe containing only the 'company_id', 'long_name' and 'growth_rate' you just calculated
    
    # Your code here.
    
    return df
    
def q_6_solution(df):
    # You are provided the dataframe.
    # The data contains a list of options represented in a variation on OSI symbology
    # 1. Using regular expressions, split the symbol into its component parts 'UnderSym', 'ExDate', 'OpType', 'Strike'
    #     (hint: you may want to use the following pattern: '\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)' )
    # 2. Cast the expiration date to datetime and the strike column to float
    #     (hint: pd.to_datetime(df['ExDate']))
    # Return the resulting dataframe.
    
    # Your code here.
    
    PATTERN= '\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)'
    
    df[['UnderSym', 'ExDate','OpType','Strike']] = df['FeedSym'].str.extract(PATTERN)
    
    df['ExDate'] = pd.to_datetime(df['ExDate'])
    df['Strike'] = df['Strike'].astype(float)
            
    return df
    
    
def q_7_solution(data):
    # Solve q_6 first.
    # Using the dataframe produced from q_6, provided to you.
    # Filter and return a dataframe containing all the options expiring after 9/30/21
    # and with strike prices between 150 and 200, inclusive
    # Return the dataframe
    
    # Your code here.
    
    df = data
    
    # reference: https://stackoverflow.com/questions/51827134/comparison-between-datetime-and-datetime64ns-in-pandas
    df["ExDate"] = pd.to_datetime(df["ExDate"]).dt.date
    
    df = df[df['ExDate']>datetime.date(2021,9,30)]
    df = df[df['Strike']>=150]
    df = df[df['Strike']<=200]
    
    return df
    
# QUESTION 8
# You will need to complete the call and put functions below.
# Please see the question description which gives you the black-scholes formula.
# These functions will be tested in q_8.
# They will also be used in q_9.
# Please round answers to 2 places.

# reference: https://www.codearmo.com/python-tutorial/options-trading-black-scholes-model

N = norm.cdf

def bs_call(S, K, T, r, sigma):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def bs_put(S, K, T, r, sigma):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)


def q_9_solution(data):
    # You will likely need to solve q_8 first, but it is not required.
    # You are provided the solution to q_7 in a dataframe.
    # Assumptions:
    #     On 10/21/22, assume all stocks are trading at $160 and all options have a constant volatility of 20%.
    #     The annual interest rate is 2%.
    #     365 days in the year.
    # 1. Calculate the BlackScholes theoretical option price for the option and put it into 'TheoPrice' column.
    #    You will need to call the appropriate function bc_call or bs_put based on row['OpType'].
    #    We are suggesting to use the function apply to answer this question.
    #    We consider the keyword 'row' being one row on the dataframe data.
    #         T = (row['ExDate'] - datetime.datetime(2021, 10, 21)).days / 365
    # 2. Round price to 2 places.
    # Return the dataframe.
    S = 160
    r = 0.02
    sigma = 0.2    
    
    # Your code here
    
    df = data
    df['TheoPrice'] = 0.0
    
    for i in range(len(df)):
        
        if df['OpType'][i]=="C":
            T = (df['ExDate'][i] - datetime.datetime(2021, 10, 21)).days / 365
            df['TheoPrice'][i] = bs_call(S, df['Strike'][i], T, r, sigma)
            # df['TheoPrice'][i] = round(df['TheoPrice'][i], 2)
            
        if df['OpType'][i]=="P":
            T = (df['ExDate'][i] - datetime.datetime(2021, 10, 21)).days / 365
            df['TheoPrice'][i] = bs_put(S, df['Strike'][i], T, r, sigma)
            # df['TheoPrice'][i] = round(df['TheoPrice'][i], 2)
    
    df.round({'TheoPrice': 2})
    
    return df

    
def q_1():
    df = pd.read_csv(sys.stdin, index_col=0, parse_dates=True) 
    df = q_1_solution(df)
    df = df.round(4)
    print(df.to_csv(line_terminator='\n'))
    

def q_2():
    df = pd.read_csv(sys.stdin, index_col=0, parse_dates=True) 
    df = q_2_solution(df)
    df = df.round(4)
    print(df.to_csv(line_terminator='\n'))
    
    
def q_3():
    df = pd.read_csv(sys.stdin)
    df = q_3_solution(df)
    print(df.to_csv(index=None, line_terminator='\n'))
    
def q_4():
    df = pd.read_csv(sys.stdin)
    df = q_4_solution(df)
    print(df.to_csv(index=None, line_terminator='\n'))
    
def q_5():
    comp = io.StringIO()
    indic = io.StringIO()
    lines = int(sys.stdin.readline().strip())
    for _ in range(lines):
        comp.write(sys.stdin.readline())
    indic.write(sys.stdin.read())
    comp.seek(0)
    indic.seek(0)
    df_comp = pd.read_csv(comp)
    df_indic = pd.read_csv(indic)
    df = q_5_solution(df_comp, df_indic)
    df = df.round(4)
    print(df.to_csv(index=None, line_terminator='\n'))
            
def q_6():
    df = pd.read_csv(sys.stdin)
    df = q_6_solution(df)
    print(df.to_csv(index=None, line_terminator='\n'))
    
def q_7():
    df = pd.read_csv(sys.stdin, parse_dates=['ExDate'])
    df = q_7_solution(df)
    print(df.to_csv(index=None, line_terminator='\n'))
    
def q_8():
    data = io.StringIO(sys.stdin.read())
    count = int(data.readline().strip())
    options = [data.readline().strip().split(',') for _ in range(count)]
    for option in options:
        option_type = option.pop(0)
        params = list(map(float, option))
        if option_type == 'C':
            print(np.round(bs_call(*params),2))
        elif option_type == 'P':
            print(np.round(bs_put(*params),2))
                
def q_9():
    df = pd.read_csv(sys.stdin, parse_dates=['ExDate'])
    df = q_9_solution(df)
    df = df.round(2)
    print(df.to_csv(index=None, line_terminator='\n'))
    

if __name__ == '__main__':
    import os
    file_size = os.path.getsize(sys.argv[0])
    sys.stderr.write(str(file_size))
    if file_size > 15000:
        raise SolutionTooLong(file_size)
    question = sys.stdin.readline().strip()
    globals()[question]()

## Q2

In [ ]:
######### Solution #########

#Q2
class Message(object):
    def __init__(self, message: str, sender: int, receiver: int) -> None:
        self.message = message
        self.sender = sender
        self.receiver = receiver

    def __str__(self) -> str:
        return self.message

    def __eq__(self, __o: object) -> bool:
        if isinstance(__o, Message):
            return __o.message == self.message
        elif isinstance(__o, str):
            return __o == self.message

In [ ]:
######### my version #########

#!/bin/python3

import math
import os
import random
import re
import sys


class Message(object):
    def __init__(self, message: str, sender: int, receiver: int) -> None:
        self.message = message
        self.sender = sender
        self.receiver = receiver

    def __str__(self) -> str:
        
        return self.message
    
    def __eq__(self, __o: object) -> bool:
        
        if self.message == __o.message:
            return True
        else:
            return False
    
        

if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    num_message = int(input())
    messages = [None]*num_message
    for i in range(num_message):
        *message, sender, receiver = input().split()
        message = " ".join(message)
        sender, receiver = map(int, [sender, receiver])
        messages[i] = Message(message, sender, receiver)

    result = ""
    num_queries = int(input())
    for i in range(num_queries):
        instruction, *ids =  input().split()
        if instruction == 'print':
            id = int(ids[0])
            result += f"{messages[id]}\n"
        elif instruction == 'eq':
            id1, id2 = map(int, ids)
            result += f"{messages[id1] == messages[id2]}\n"
    fptr.write(result + '\n')

    fptr.close()
